In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [3]:
file_ = os.listdir('./temp_data/')
stock_df = pd.DataFrame()
for i in file_:
    temp = pd.read_csv(f'./temp_data/{i}')
    
    stock_df = pd.concat([stock_df, temp], axis=0)

In [4]:
# 첫 5줄을 봅니다.
stock_df

,날짜,시간,시가,고가,저가,종가,거래량,거래대금,누적체결매수수량,누적체결매도수량,...,시가총액,외국인주문한도수량,외국인주문가능수량,외국인현보유수량,외국인현보유비율,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,pct_label
0,20221121,1530,8920.0,8920.0,8920.0,8920.0,4128.0,36820000.0,21105.0,101675.0,...,249144000000,27931470,26667255,1264215,4.53,20221121,100.0,-8301,83679,0.0
1,20221121,1520,8920.0,8920.0,8920.0,8920.0,605.0,5390000.0,16977.0,101675.0,...,249144000000,27931470,26667255,1264215,4.53,20221121,100.0,-8301,83679,0.0
2,20221121,1519,8920.0,8920.0,8920.0,8920.0,323.0,2880000.0,16977.0,101070.0,...,249144000000,27931470,26667255,1264215,4.53,20221121,100.0,-8301,83679,0.0
3,20221121,1518,8920.0,8920.0,8920.0,8920.0,734.0,6550000.0,16977.0,100747.0,...,249144000000,27931470,26667255,1264215,4.53,20221121,100.0,-8301,83679,0.0
4,20221121,1517,8920.0,8920.0,8920.0,8920.0,316.0,2820000.0,16977.0,100013.0,...,249144000000,27931470,26667255,1264215,4.53,20221121,100.0,-8301,83679,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232666,20201028,905,14500.0,14500.0,14500.0,14500.0,120.0,1740000.0,3026.0,831.0,...,1693832000000,113680000,100720528,12959472,11.40,20201028,100.0,24356,-2160796,3.1
232667,20201028,904,14500.0,14550.0,14500.0,14500.0,442.0,6410000.0,3026.0,711.0,...,1693832000000,113680000,100720528,12959472,11.40,20201028,100.0,24356,-2160796,2.7
232668,20201028,903,14550.0,14550.0,14550.0,14550.0,418.0,6080000.0,3025.0,270.0,...,1693832000000,113680000,100720528,12959472,11.40,20201028,100.0,24356,-2160796,2.7
232669,20201028,902,14550.0,14550.0,14450.0,14550.0,1436.0,20820000.0,2607.0,270.0,...,1693832000000,113680000,100720528,12959472,11.40,20201028,100.0,24356,-2160796,2.7


In [5]:
stock_df.corr()

,날짜,시간,시가,고가,저가,종가,거래량,거래대금,누적체결매수수량,누적체결매도수량,...,시가총액,외국인주문한도수량,외국인주문가능수량,외국인현보유수량,외국인현보유비율,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,pct_label
날짜,1.000000,-0.003349,0.132412,0.132232,0.132607,0.132430,-0.040533,-0.033185,-0.069726,-0.079541,...,0.152500,0.003820,-0.000092,0.009022,-0.000361,1.000000,NaN,0.011094,0.226157,-0.020186
시간,-0.003349,1.000000,-0.000454,-0.000554,-0.000341,-0.000468,-0.029031,-0.008319,0.064916,0.077947,...,-0.000423,-0.000041,0.000004,-0.000104,-0.000062,-0.003349,NaN,-0.000900,-0.000029,-0.211943
시가,0.132412,-0.000454,1.000000,0.999996,0.999996,0.999994,-0.056513,0.059150,-0.108192,-0.128053,...,0.903939,0.231703,0.558886,-0.492300,-0.565031,0.132412,NaN,0.083805,0.873680,-0.006228
고가,0.132232,-0.000554,0.999996,1.000000,0.999994,0.999997,-0.056292,0.059572,-0.108078,-0.127939,...,0.903982,0.231864,0.559048,-0.492225,-0.564970,0.132232,NaN,0.083683,0.873693,-0.006125
저가,0.132607,-0.000341,0.999996,0.999994,1.000000,0.999996,-0.056717,0.058777,-0.108342,-0.128206,...,0.903888,0.231506,0.558711,-0.492434,-0.565155,0.132607,NaN,0.083829,0.873637,-0.006328
종가,0.132430,-0.000468,0.999994,0.999997,0.999996,1.000000,-0.056497,0.059195,-0.108218,-0.128084,...,0.903887,0.231575,0.558782,-0.492406,-0.565130,0.132430,NaN,0.083803,0.873637,-0.006195
거래량,-0.040533,-0.029031,-0.056513,-0.056292,-0.056717,-0.056497,1.000000,0.411424,0.345263,0.313530,...,-0.033378,0.037716,-0.000862,0.089001,0.091646,-0.040533,NaN,0.046998,-0.048189,0.097524
거래대금,-0.033185,-0.008319,0.059150,0.059572,0.058777,0.059195,0.411424,1.000000,0.095202,0.084269,...,0.042041,0.009155,0.023381,-0.021841,-0.024526,-0.033185,NaN,-0.001778,0.040072,0.043485
누적체결매수수량,-0.069726,0.064916,-0.108192,-0.108078,-0.108342,-0.108218,0.345263,0.095202,1.000000,0.984628,...,-0.064382,0.068313,-0.003648,0.165047,0.169912,-0.069726,NaN,0.069625,-0.091046,0.091749
누적체결매도수량,-0.079541,0.077947,-0.128053,-0.127939,-0.128206,-0.128084,0.313530,0.084269,0.984628,1.000000,...,-0.077086,0.077206,-0.006587,0.191070,0.197089,-0.079541,NaN,0.042115,-0.108898,0.089715


In [6]:

# stock_df['pct_label'] = stock_df['pct_label'].mul(100)
stock_df['pct_label'] = stock_df['pct_label'].apply(np.floor)
# stock_df

In [7]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
# stock_df = stock_df.drop(columns=['상장주식수', '시가총액', '외국인주문한도수량', '외국인주문가능수량', '외국인현보유수량', '외국인현보유비율', '수정주가일자', '수정주가비율', '기관순매수량', '기관누적순매수량']
#             , axis=1
# )

X_stock_df = stock_df.iloc[:,0:-1]
y_stock_df = stock_df.iloc[:,-1]


In [8]:
from sklearn.model_selection import train_test_split

X_stock_sc = min_max_scaler.fit_transform(X_stock_df)

X_stock_trans_df = pd.DataFrame(data=X_stock_sc, columns=X_stock_df.columns)



In [9]:
X_stock_trans_df

,날짜,시간,시가,고가,저가,종가,거래량,거래대금,누적체결매수수량,누적체결매도수량,...,상장주식수,시가총액,외국인주문한도수량,외국인주문가능수량,외국인현보유수량,외국인현보유비율,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량
0,1.0,0.862826,0.158947,0.158916,0.159629,0.158995,0.001052,0.000292,0.013612,2.179984e-03,...,0.005536,0.031737,0.005538,0.006251,0.021479,0.074398,1.0,0.0,0.613076,0.322575
1,1.0,0.849108,0.158947,0.158916,0.159629,0.158995,0.000154,0.000043,0.013524,2.179984e-03,...,0.005536,0.031737,0.005538,0.006251,0.021479,0.074398,1.0,0.0,0.613076,0.322575
2,1.0,0.847737,0.158947,0.158916,0.159629,0.158995,0.000082,0.000023,0.013524,2.167013e-03,...,0.005536,0.031737,0.005538,0.006251,0.021479,0.074398,1.0,0.0,0.613076,0.322575
3,1.0,0.846365,0.158947,0.158916,0.159629,0.158995,0.000187,0.000052,0.013524,2.160087e-03,...,0.005536,0.031737,0.005538,0.006251,0.021479,0.074398,1.0,0.0,0.613076,0.322575
4,1.0,0.844993,0.158947,0.158916,0.159629,0.158995,0.000081,0.000022,0.013524,2.144350e-03,...,0.005536,0.031737,0.005538,0.006251,0.021479,0.074398,1.0,0.0,0.613076,0.322575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930679,0.0,0.005487,0.264220,0.264193,0.265291,0.264262,0.000031,0.000014,0.013227,1.781723e-05,...,0.925491,0.260311,0.925491,0.902075,0.284782,0.224726,0.0,0.0,0.630775,0.053423
930680,0.0,0.004115,0.264220,0.265136,0.265291,0.264262,0.000113,0.000051,0.013227,1.524435e-05,...,0.925491,0.260311,0.925491,0.902075,0.284782,0.224726,0.0,0.0,0.630775,0.053423
930681,0.0,0.002743,0.265164,0.265136,0.266237,0.265205,0.000107,0.000048,0.013227,5.788992e-06,...,0.925491,0.260311,0.925491,0.902075,0.284782,0.224726,0.0,0.0,0.630775,0.053423
930682,0.0,0.001372,0.265164,0.265136,0.264344,0.265205,0.000366,0.000165,0.013218,5.788992e-06,...,0.925491,0.260311,0.925491,0.902075,0.284782,0.224726,0.0,0.0,0.630775,0.053423


In [10]:
y_stock_df.value_counts()

0.0     650132
1.0     175131
2.0      54515
3.0      21264
4.0       9842
5.0       5975
6.0       3949
7.0       2256
8.0       1566
9.0       1290
11.0       735
10.0       724
12.0       600
13.0       359
15.0       225
14.0       213
26.0       211
24.0       201
22.0       200
20.0       195
23.0       130
19.0       129
21.0       126
25.0       123
30.0       119
18.0        95
16.0        91
29.0        63
17.0        60
31.0        56
27.0        50
28.0        19
37.0        12
32.0         6
34.0         4
41.0         4
40.0         4
35.0         2
42.0         2
38.0         2
33.0         2
36.0         2
Name: pct_label, dtype: int64

In [11]:
y_stock_df = to_categorical(y_stock_df)

In [12]:
y_stock_df.shape

(930684, 43)

In [13]:
X_stock_df = X_stock_trans_df.values.tolist()

In [14]:
X_stock_df[:5]

[[1.0,
  0.8628257887517148,
  0.15894726912555418,
  0.15891553308303302,
  0.159628858170801,
  0.15899486869906426,
  0.0010524504273667706,
  0.0002918764875732515,
  0.013612455007463019,
  0.002179984307114688,
  0.5113924050632912,
  0.005535886707434823,
  0.03173745038130981,
  0.005538048441938392,
  0.006251233896441721,
  0.02147941211505718,
  0.07439825115962136,
  1.0,
  0.0,
  0.613076236858564,
  0.32257495874232256],
 [1.0,
  0.8491083676268862,
  0.15894726912555418,
  0.15891553308303302,
  0.159628858170801,
  0.15899486869906426,
  0.00015424721622017833,
  4.272716643182579e-05,
  0.013524447855858612,
  0.002179984307114688,
  0.5113924050632912,
  0.005535886707434823,
  0.03173745038130981,
  0.005538048441938392,
  0.006251233896441721,
  0.02147941211505718,
  0.07439825115962136,
  1.0,
  0.0,
  0.613076236858564,
  0.32257495874232256],
 [1.0,
  0.8477366255144034,
  0.15894726912555418,
  0.15891553308303302,
  0.159628858170801,
  0.15899486869906426,
  

In [15]:
window_size = 10

data_X = []
data_y = []
for i in range(len(y_stock_df) - window_size):
    _X = X_stock_df[i : i + window_size] # 다음 날 종가(i+windows_size)는 포함되지 않음
    _y = y_stock_df[i + window_size]     # 다음 날 종가
    data_X.append(_X)
    data_y.append(_y)
print(_X, "->", _y)

[[0.0, 0.013717421124828544, 0.26233374209980187, 0.2632492500424505, 0.2633970838856277, 0.2633187443404769, 0.0008194224015399226, 0.00036797682110674456, 0.013286287029653954, 0.00024120800054133507, 0.5759493670886076, 0.9254908271644674, 0.26031101261603007, 0.9254906113342458, 0.9020754501060668, 0.2847817114309388, 0.22472645975362418, 0.0, 0.0, 0.6307753758838188, 0.05342272636680373], [0.0, 0.012345679012345734, 0.2642203565701349, 0.26513593570175276, 0.2633970838856277, 0.2623754904920012, 0.002106430579191923, 0.0009468942542266402, 0.013228809490718517, 0.00023010171216085405, 0.5759493670886076, 0.9254908271644674, 0.26031101261603007, 0.9254906113342458, 0.9020754501060668, 0.2847817114309388, 0.22472645975362418, 0.0, 0.0, 0.6307753758838188, 0.05342272636680373], [0.0, 0.010973936899862924, 0.2642203565701349, 0.2641925928721016, 0.2652906646468472, 0.26426199818895263, 0.0001575616192133392, 7.102697796459352e-05, 0.013228468377727802, 5.330160794184524e-05, 0.5759493

In [16]:
# X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.2, shuffle=True, random_state=42)

In [17]:
train_size = int(len(data_y) * 0.7)
train_X = np.array(data_X[0 : train_size])
train_y = np.array(data_y[0 : train_size])

test_size = len(data_y) - train_size
test_X = np.array(data_X[train_size : len(data_X)])
test_y = np.array(data_y[train_size : len(data_y)])

print('훈련 데이터의 크기 :', train_X.shape, train_y.shape)
print('테스트 데이터의 크기 :', test_X.shape, test_y.shape)

훈련 데이터의 크기 : (651471, 10, 21) (651471, 43)
테스트 데이터의 크기 : (279203, 10, 21) (279203, 43)


In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

# X_train_lstm = X_train.reshape(X_train.shape[0], 21, 1)
# X_test_lstm = X_test.reshape(X_test.shape[0], 21, 1)


In [19]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [25]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout, PReLU, LeakyReLU
from tensorflow.keras.activations import relu, tanh
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import optimizers
# 모델 설정
model = Sequential()
model.add(LSTM(210, input_shape=(10, 21), return_sequences=True, activation='PReLU'))
# model.add(Dropout(0.2))
model.add(LSTM(100, activation='PReLU'))
model.add(Dropout(0.2))
model.add(Dense(43, activation='softmax')) # 답이랑 오차를 구하기 위해서 값의 총합을 1로 바꾸는 함수
model.summary()
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)
# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

# 모델 실행
history=model.fit(train_X, train_y, epochs=50, batch_size=500,verbose=1, validation_data=(test_X, test_y), callbacks=[early_stopping_callback])
print("\n Test Accuracy: %.4f" % (model.evaluate(test_X, test_y)[1]))


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 10, 210)           195090    
                                                                 
 lstm_9 (LSTM)               (None, 100)               124500    
                                                                 
 dropout_5 (Dropout)         (None, 100)               0         
                                                                 
 dense_4 (Dense)             (None, 43)                4343      
                                                                 
Total params: 323,933
Trainable params: 323,933
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
1303/1303 [==============================] - 168s 128ms/step - loss: 1.1155 - accuracy: 0.6992 - val_loss: 1.5268 - val_accuracy: 0.4753
Epoch 2/50
1303/1303 [==========

KeyboardInterrupt: 

In [ ]:
model.evaluate(X_test, y_test)[1]

0.7138068079948425

In [ ]:
predict = model.predict(X_test)

1455/1455 [==============================] - 5s 3ms/step


In [ ]:
predict[0]

array([6.3224030e-01, 2.6978216e-01, 6.0484692e-02, 2.6280599e-02,
       5.7714609e-03, 2.4538008e-03, 1.3107358e-03, 6.4932340e-04,
       4.7613916e-04, 8.6917702e-05, 1.0630668e-04, 5.9439029e-05,
       2.8661638e-04, 3.1551381e-06, 1.1152962e-06, 5.4803769e-07,
       1.0371655e-07, 2.6900693e-07, 9.2528481e-08, 1.8171394e-07,
       1.5282910e-06, 9.5656173e-07, 6.7619561e-07, 6.5209758e-07,
       5.3779348e-07, 3.9681004e-07, 9.8080150e-08, 8.2230827e-07,
       4.4638597e-07, 2.3792190e-08], dtype=float32)

In [ ]:
predict.index(predict[0][0].max())

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [ ]:
# for i,rate  in enumerate(predict):
#     for i, ra in enumerate(rate):
#         if rate.max() == ra:
#             print(i)

# print(list(map(np.argmax, predict)))